In [3]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection
import ccdproc as ccdp

from astropy.nddata import CCDData
from astropy.stats import mad_std
import astropy.units as u
from convenience_functions import show_image

# suppress warnings is not recommended :I
import warnings
warnings.filterwarnings("ignore")

# Subtract Bias

In [8]:
exp_time = 900.0

bias_dir = './data/bias/'
dark_dir = './data/dark/'
combined_bias = CCDData.read(bias_dir + '/combined_bias.fits')
raw_darks = ImageFileCollection(dark_dir + 'raw')

for ccd, file_name in raw_darks.ccds(
    imagetyp='DARK',
    exptime=exp_time,
    return_fname=True,
    unit='adu'
):
    processed = ccdp.subtract_bias(ccd, combined_bias)
    ccd.write(dark_dir + f'reduced/{file_name}', overwrite=True)

# Combine Darks

In [10]:
exp_time = 900.0

reduced_darks = ImageFileCollection(dark_dir + 'reduced')
combined_dark = ccdp.combine(
    reduced_darks.files_filtered(imagetyp='DARK', include_path=True),
    method='average',
    sigma_clip=True,
    sigma_clip_low_thresh=5,
    sigma_clip_high_thresh=5,
    sigma_clip_func=np.ma.median,
    sigma_clip_dev_func=mad_std,
    mem_limit=350e+6
)

combined_dark.meta['combined'] = True
dark_file_name = 'combined_dark_{:6.3f}.fits'.format(exp_time)
combined_dark.write(dark_dir + f'/{dark_file_name}', overwrite=True)

INFO:astropy:splitting each image into 5 chunks to limit memory usage to 350000000.0 bytes.


INFO: splitting each image into 5 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]
